## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,35.26,87,40,26.46,scattered clouds
1,1,Aljezur,PT,37.3191,-8.8033,65.71,96,35,9.17,scattered clouds
2,2,Mayo,US,38.8876,-76.5119,90.52,72,20,3.00,few clouds
3,3,Hilo,US,19.7297,-155.0900,81.81,83,90,6.91,moderate rain
4,4,Southbridge,US,42.0751,-72.0334,76.93,92,99,7.70,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature acceptable for your trip? "))
max_temp = float(input("What is the maximum temperature acceptable for your trip"))

What is the minimum temperature acceptable for your trip? 60
What is the maximum temperature acceptable for your trip85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
acceptable_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
acceptable_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Aljezur,PT,37.3191,-8.8033,65.71,96,35,9.17,scattered clouds
3,3,Hilo,US,19.7297,-155.0900,81.81,83,90,6.91,moderate rain
4,4,Southbridge,US,42.0751,-72.0334,76.93,92,99,7.70,overcast clouds
6,6,Rikitea,PF,-23.1203,-134.9692,71.01,67,19,4.70,few clouds
7,7,Muros,ES,42.7762,-9.0603,65.98,84,8,6.40,clear sky
8,8,Port Alfred,ZA,-33.5906,26.8910,61.16,37,65,10.27,broken clouds
10,10,Lakota,CI,5.8528,-5.6883,69.73,96,70,2.84,broken clouds
11,11,Puerto Carreno,CO,6.1890,-67.4859,83.44,71,99,1.90,overcast clouds
12,12,Harper,LR,4.3750,-7.7169,75.49,76,22,6.24,few clouds
16,16,Miraflores,MX,19.2167,-98.8083,78.03,91,82,3.36,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
acceptable_cities_df.count()

Unnamed: 0             415
City                   415
Country                412
Lat                    415
Lng                    415
Max Temp               415
Humidity               415
Cloudiness             415
Wind Speed             415
Current Description    415
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
acceptable_cities_df = acceptable_cities_df.dropna()
acceptable_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Aljezur,PT,37.3191,-8.8033,65.71,96,35,9.17,scattered clouds
3,3,Hilo,US,19.7297,-155.0900,81.81,83,90,6.91,moderate rain
4,4,Southbridge,US,42.0751,-72.0334,76.93,92,99,7.70,overcast clouds
6,6,Rikitea,PF,-23.1203,-134.9692,71.01,67,19,4.70,few clouds
7,7,Muros,ES,42.7762,-9.0603,65.98,84,8,6.40,clear sky
8,8,Port Alfred,ZA,-33.5906,26.8910,61.16,37,65,10.27,broken clouds
10,10,Lakota,CI,5.8528,-5.6883,69.73,96,70,2.84,broken clouds
11,11,Puerto Carreno,CO,6.1890,-67.4859,83.44,71,99,1.90,overcast clouds
12,12,Harper,LR,4.3750,-7.7169,75.49,76,22,6.24,few clouds
16,16,Miraflores,MX,19.2167,-98.8083,78.03,91,82,3.36,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = acceptable_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Aljezur,PT,65.71,scattered clouds,37.3191,-8.8033,
3,Hilo,US,81.81,moderate rain,19.7297,-155.0900,
4,Southbridge,US,76.93,overcast clouds,42.0751,-72.0334,
6,Rikitea,PF,71.01,few clouds,-23.1203,-134.9692,
7,Muros,ES,65.98,clear sky,42.7762,-9.0603,
8,Port Alfred,ZA,61.16,broken clouds,-33.5906,26.8910,
10,Lakota,CI,69.73,broken clouds,5.8528,-5.6883,
11,Puerto Carreno,CO,83.44,overcast clouds,6.1890,-67.4859,
12,Harper,LR,75.49,few clouds,4.3750,-7.7169,
16,Miraflores,MX,78.03,broken clouds,19.2167,-98.8083,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        pass
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Aljezur,PT,65.71,scattered clouds,37.3191,-8.8033,Amazigh Hostel
3,Hilo,US,81.81,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
4,Southbridge,US,76.93,overcast clouds,42.0751,-72.0334,Southbridge Hotel & Conference Center
6,Rikitea,PF,71.01,few clouds,-23.1203,-134.9692,People ThankYou
7,Muros,ES,65.98,clear sky,42.7762,-9.0603,Apartment Av. Castelao
...,...,...,...,...,...,...,...
675,Udachnyy,RU,72.45,broken clouds,66.4167,112.4000,Severnoye Siyaniye
678,Chiang Rai,TH,75.18,light rain,19.9167,99.9167,Bansaeo Garden and Resort
680,Huangmei,CN,76.48,broken clouds,30.1924,116.0250,Tianxiachan Hotel
684,Denpasar,ID,73.06,scattered clouds,-8.6500,115.2167,ASTON Denpasar Hotel & Convention Center


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Aljezur,PT,65.71,scattered clouds,37.3191,-8.8033,Amazigh Hostel
3,Hilo,US,81.81,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
4,Southbridge,US,76.93,overcast clouds,42.0751,-72.0334,Southbridge Hotel & Conference Center
6,Rikitea,PF,71.01,few clouds,-23.1203,-134.9692,People ThankYou
7,Muros,ES,65.98,clear sky,42.7762,-9.0603,Apartment Av. Castelao
8,Port Alfred,ZA,61.16,broken clouds,-33.5906,26.8910,The Halyards Hotel
10,Lakota,CI,69.73,broken clouds,5.8528,-5.6883,Aziz bah divers
11,Puerto Carreno,CO,83.44,overcast clouds,6.1890,-67.4859,Casa Hotel Victoria
12,Harper,LR,75.49,few clouds,4.3750,-7.7169,Screensaver
16,Miraflores,MX,78.03,broken clouds,19.2167,-98.8083,Hotel Mansión


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))